In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

In [2]:
from tqdm import tqdm

In [3]:
from SentimentClassifier import TwoStepSentimentClassifier
classifier = TwoStepSentimentClassifier()

Init vader
Init DistilBERT SST-2


Couldn't reach server at 'https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-finetuned-sst-2-english-modelcard.json' to download model card file.
Creating an empty model card.


In [4]:
# initiate random seed
np.random.seed(4242)

In [5]:
data_path = os.path.join(os.getcwd(), "STEP 1 - RAW_JSON_TO_CSV")

In [6]:
mask = "$T$"

In [7]:
def apply_sentiment(row):
    text = row["text"]
    restored_text = text.replace(mask, row["name"])
    sent = classifier.label_and_confidence(restored_text)
    row["sentiment"] = sent.label
    row["sentiment_score"] = sent.confidence
    return row

In [8]:
def make_path(root, folder):
    path = os.path.join(root, folder)
    if not os.path.exists(path):
        os.mkdir(path)
    return path

In [9]:
store_path = os.path.join(os.getcwd(), "STEP 2 - CSV_WITH_SENTIMENT")
store_path

'C:\\Users\\Tollef\\Documents\\GitHub\\masterNEW\\REPO\\strise\\STEP 2 - CSV_WITH_SENTIMENT'

# Vader threshold 0.12

In [10]:
tqdm.pandas()

c:\users\tollef\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [11]:
all_topics_entity_dfs = []
all_topics_texts_dfs = []

for subdir, dirs, files in os.walk(data_path):
    folder_name = os.path.split(subdir)[-1] if not dirs else None
    if not folder_name:
        continue
        
    topic_files = [f for f in files if '_' in f]
    for csv in topic_files:
        csv_path = os.path.join(subdir, csv)
        print("reading", csv_path)

        ent_df = pd.read_csv(csv_path)
        print(ent_df.shape)
        
        ent_df.drop_duplicates(subset="text", inplace=True)
        print("drop dups")
        print(ent_df.shape)
        
        ent_df = ent_df.dropna()
        print("dropping NaN")
        print(ent_df.shape)
        
        ent_df["sentiment"] = np.nan
        ent_df["sentiment_score"] = np.nan
        
        # use tqdm's progress_apply instead of apply
        ent_df = ent_df.progress_apply(lambda row: apply_sentiment(row), axis=1)
        
        all_topics_entity_dfs.append(ent_df)
        
        new_path = os.path.join(store_path, csv)
        print("storing in", new_path)
        ent_df.to_csv(new_path)


  0%|                                                  | 3/1627 [00:00<01:09, 23.44it/s]

reading C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 1 - RAW_JSON_TO_CSV\business_entities.csv
(1713, 6)
drop dups
(1627, 6)
dropping NaN
(1627, 6)


  0%|                                                          | 0/1683 [00:00<?, ?it/s]

storing in C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 2 - CSV_WITH_SENTIMENT\business_entities.csv
reading C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 1 - RAW_JSON_TO_CSV\politics_entities.csv
(1746, 6)
drop dups
(1683, 6)
dropping NaN
(1683, 6)


  0%|                                                          | 0/1551 [00:00<?, ?it/s]

storing in C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 2 - CSV_WITH_SENTIMENT\politics_entities.csv
reading C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 1 - RAW_JSON_TO_CSV\sports_entities.csv
(1559, 6)
drop dups
(1551, 6)
dropping NaN
(1551, 6)


  0%|                                                          | 0/1561 [00:00<?, ?it/s]

storing in C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 2 - CSV_WITH_SENTIMENT\sports_entities.csv
reading C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 1 - RAW_JSON_TO_CSV\tech_entities.csv
(1612, 6)
drop dups
(1561, 6)
dropping NaN
(1561, 6)


100%|███████████████████████████████████████████████| 1561/1561 [01:26<00:00, 18.13it/s]

storing in C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 2 - CSV_WITH_SENTIMENT\tech_entities.csv


In [12]:
# finally, merge all the entities and store it in a singular csv
final_entity_df = pd.concat(all_topics_entity_dfs)
entity_path = os.path.join(store_path, "entities.csv")
final_entity_df.to_csv(entity_path)

In [13]:
len(all_topics_entity_dfs)

4

In [ ]:
dups = None
big = None
for subdir, dirs, files in os.walk(data_path):
    folder_name = os.path.split(subdir)[-1] if not dirs else None
    if not folder_name:
        continue
        
    topic_files = [f for f in files if '_' in f]
    for csv in topic_files:
        csv_path = os.path.join(subdir, csv)
        if "sport" not in csv:
            continue
        print("reading", csv_path)

        ent_df = pd.read_csv(csv_path)
        print(ent_df.shape)
        big = ent_df
        
        print("exposing dplicates...")
        dups = ent_df.duplicated(subset="text", keep="first")
        break
        
        ent_df.drop_duplicates(subset="text", inplace=True)
        print("drop dups")
        print(ent_df.shape)
        
        ent_df = ent_df.dropna()
        print("dropping NaN")
        print(ent_df.shape)
        
        break
    break
        


In [ ]:
dups

In [ ]:
big

In [ ]:
duped = big[big.duplicated(subset=["text", "name"])]
duped.describe()

In [ ]:
dtexts = duped.text
len(dtexts)

In [ ]:
[x for x in dtexts[0:50]]

In [ ]:
cleaned = big.reset_index().drop_duplicates(big.index.text)
cleaned.set_index(big.index.names, inplace=True)

In [ ]:
big.reset_index().drop_duplicates(subset="references", keep="first")

In [ ]:
test = big[0:100]
test

In [ ]:
test.drop_duplicates(subset="text", keep="first")